<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [ ]:
import numpy as np

In [ ]:
import math

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [ ]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [ ]:
def lista_de_terminos_de_doc(entrada):
  output = entrada.split()
  return output


In [ ]:
prueba1 =lista_de_terminos_de_doc(corpus[0])
prueba1 

['que', 'dia', 'es', 'hoy']

In [ ]:
type(prueba1)

list

In [ ]:
def vectorizacion(corpus):
  lista_de_terminos_del_corpus = []
  for i in corpus:
    lista_de_terminos_del_corpus.append(lista_de_terminos_de_doc(i))
  return lista_de_terminos_del_corpus  

In [ ]:
prueba = vectorizacion(corpus)

In [ ]:
prueba

[['que', 'dia', 'es', 'hoy'],
 ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'],
 ['martes', 'muchas', 'gracias']]

In [ ]:
len(prueba)

3

In [ ]:
lista = []
for i in range(len(prueba)):
  for u in prueba[i]:
    if lista.count(u) ==0 :
      lista.append(u)


In [ ]:
lista

['que', 'dia', 'es', 'hoy', 'martes', 'el', 'de', 'muchas', 'gracias']

#
Respuesta:

In [ ]:
def vocabulario(corpus):
  lista_de_terminos_del_corpus = vectorizacion(corpus)
  lista = []
  for i in range(len(lista_de_terminos_del_corpus)):
    for u in lista_de_terminos_del_corpus[i]:
      if lista.count(u) ==0 :
        lista.append(u)
  return lista

In [ ]:
vocabulario(corpus)

['que', 'dia', 'es', 'hoy', 'martes', 'el', 'de', 'muchas', 'gracias']

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [ ]:
corpus

array(['que dia es hoy', 'martes el dia de hoy es martes',
       'martes muchas gracias'], dtype='<U30')

In [ ]:
def ohencoding(corpus):
  vector = vectorizacion(corpus)
  voc = vocabulario(corpus)
  matriz = np.empty((len(vector), len(voc)), int)
  for i in range(len(vector)):
    for u in range(len(voc)):
      matriz[i][u] = vector[i].count(voc[u])>0
  return matriz

In [ ]:
ohencoding(corpus)

array([[1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 1]])

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [ ]:
def frecuencia(corpus):
  vector = vectorizacion(corpus)
  voc = vocabulario(corpus)
  matriz = np.empty((len(vector), len(voc)), int)
  for i in range(len(vector)):
    for u in range(len(voc)):
      matriz[i][u] = vector[i].count(voc[u])
  return matriz

In [ ]:
frecuencia(corpus)

array([[1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 2, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 1]])

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [ ]:
np.sum(ohencoding(corpus), axis=0)

array([1, 2, 2, 2, 2, 1, 1, 1, 1])

In [ ]:
def idf(corpus):
  suma = np.sum(ohencoding(corpus), axis=0)
  idf = []
  for i in range(len(suma)):
    idf.append(math.log((len(corpus)/suma[i]) , 10)) 
  return idf  

In [ ]:
idf(corpus)

[0.47712125471966244,
 0.17609125905568124,
 0.17609125905568124,
 0.17609125905568124,
 0.17609125905568124,
 0.47712125471966244,
 0.47712125471966244,
 0.47712125471966244,
 0.47712125471966244]

In [ ]:
def tfidf(corpus):
  return np.multiply(frecuencia(corpus), idf(corpus))

In [ ]:
tfidf(corpus)

array([[0.47712125, 0.17609126, 0.17609126, 0.17609126, 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.17609126, 0.17609126, 0.17609126, 0.35218252,
        0.47712125, 0.47712125, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.17609126,
        0.        , 0.        , 0.47712125, 0.47712125]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [ ]:
def orden_sim_cos(corpus, index):
  matriz = frecuencia(corpus) ## ohencoding o tfidf
  lista = [0]
  for doc_del_corpus in range(1, len(corpus)):
    insertado = False
    index_en_salida = 0
    while insertado == False:
      if index_en_salida < len(lista): 
        if cosine_similarity(matriz[doc_del_corpus], matriz[index]) > cosine_similarity(matriz[lista[index_en_salida]], matriz[index]):
          lista.insert(index_en_salida, doc_del_corpus)
          insertado = True
        else:
          index_en_salida = index_en_salida+1
      else:
        lista.append(doc_del_corpus)
        insertado = True
  for i in range(len(lista)):
    lista[i] = corpus[lista[i]]
  return lista 

In [ ]:
corpus = np.append(corpus,'hoy es dia martes')
corpus = np.append(corpus,'es martes de dia')
corpus = np.append(corpus,'que buen dia este martes')
corpus = np.append(corpus,'no me gustan los martes')

In [ ]:
corpus

array(['que dia es hoy', 'martes el dia de hoy es martes',
       'martes muchas gracias', 'hoy es dia martes', 'es martes de dia',
       'que buen dia este martes', 'no me gustan los martes'],
      dtype='<U30')

In [ ]:
frec = frecuencia(corpus)
frec

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]])

pruebas comparando vectores de frecuencia en orden_sim_cos


In [ ]:
prueba = orden_sim_cos(corpus, 3)
for i in range(7):
  print(cosine_similarity(frecuencia(prueba)[i], frecuencia(prueba)[0]))

1.0
0.8333333333333334
0.75
0.75
0.4472135954999579
0.2886751345948129
0.22360679774997896


In [ ]:
prueba

['hoy es dia martes',
 'martes el dia de hoy es martes',
 'que dia es hoy',
 'es martes de dia',
 'que buen dia este martes',
 'martes muchas gracias',
 'no me gustan los martes']

pruebas comparando vectores de one hot encoding en orden_sim_cos

In [ ]:
prueba = orden_sim_cos(corpus, 3)
for i in range(7):
  print(cosine_similarity(ohencoding(prueba)[i], ohencoding(prueba)[0]))

1.0
0.8164965809277261
0.75
0.75
0.4472135954999579
0.2886751345948129
0.22360679774997896


In [ ]:
prueba = orden_sim_cos(corpus, 3)
prueba

['hoy es dia martes',
 'martes el dia de hoy es martes',
 'que dia es hoy',
 'es martes de dia',
 'que buen dia este martes',
 'martes muchas gracias',
 'no me gustan los martes']

pruebas comparando vectores de tfidf en orden_sim_cos

In [ ]:
prueba = orden_sim_cos(corpus, 3)
for i in range(7):
  print(cosine_similarity(tfidf(prueba)[i], tfidf(prueba)[0]))

1.0
0.6427238470614614
0.4293986595497244
0.29308410982958116
0.041605347163416434
0.007977013808851859
0.0056450165663072165


In [ ]:
prueba

['hoy es dia martes',
 'que dia es hoy',
 'martes el dia de hoy es martes',
 'es martes de dia',
 'que buen dia este martes',
 'martes muchas gracias',
 'no me gustan los martes']

Los valores de similitud del coseno dan distinto dependiendo de que vectores (frecuencia , OHE o tfidf) se usen